### I590: Python - Professor Vel Malbasa (Spring 2017) 

### Assignment 10: PANDAS
#### Sean Shiverick               March 26, 2017

### Question 1 (60 points) 
* Write function invest(d, m, n), which implements the investment algorithm described below.
* Use function invest(d, m, n) to determine the values m and n which will produce the best return of your initial investment of 10,000 in company Microsoft, in year 2015.

Function main():
* Prompts the user for the ticker symbol (an abbreviation used to uniquely identify publicly traded shares of a particular stock on NYSE).
* Reads the Microsoft stock data from Yahoo!Finances into variable d.
* Calls function invest(d, m, n) for all combinations of m, n ϵ {1, 2, 3}
* Collects the balance returned by the function invest into a two dimensional array or list with columns m, n, balance.
* Prints the results in the form:
```
    m   n   No. transactions   End balance
0   1   1   20                 10930.8252745
1   1   2   . . .              . . .
2   1   3   . . .              . . .
3   2   1   . . .              . . .
4   2   2   . . .              . . .
5   2   3   . . .              . . .
6   3   1   . . .              . . .
7   3   2   . . .              . . .
8   3   3   . . .              . . .
```
Function invest(d, m, n):
• Implements the investment algorithm.
• Returns the value of the investment at the end of the investment period.

You may want to design and implement any other function that you find appropriate to make your solution more readable, elegant and efficient.

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import pandas_datareader as pdr 
from pandas_datareader import data, wb

def invest(d, m, n):
    #Implements the investment algorithm, starting with capital of 10000
    capital = 10000    
    # Charged seven dollar fee on each transaction (buy / sell). 
    transaction_fee = 7    
    # Extract the closing values indexed by the date:
    d_Close = d["Close"]
    buying_price = d_Close[0]
    
    # Invest complete balance into X shares at beginning of time period.
    # Before you buy, first subtract transaction fee, then 
    # use resulting balance to buy as many shares as possible.    
    bought_shares = (capital - transaction_fee) / buying_price
    money_left = 0
    # Start transaction on second day
    tx = 1                 
    streak = 0
    yesterday = buying_price

    # Use for loop / iterate from second day w/ current iteration closing price    
    # for today in d_Close[1:]: 
    for index, row in d.iloc[1:, :].iterrows():  
        today = row[3]
        #days_down = 0
        #days_up = 0
        
        if today > yesterday and streak >= 0:
            streak += 1 
        elif today < yesterday and streak <= 0:
            streak -= 1       
        else:
            streak = 0
    
        if bought_shares > 0 and streak <= m:
            # check if stocks are down m consecutive days, and sell all shares.
            # After sell, subtract transaction fee from sell value to get balance.
            money_left += (bought_shares * today)-7
            tx = tx + 1
            bought_shares = 0
        
        elif bought_shares == 0 and streak >= n:
            # Check if stock is up m consecutive days, buy as many shares
            # If buy, subtract fee, buy as many shares as possible with balance.
            bought_shares += (money_left - 7) / today
            tx = tx + 1
            money_left = 0

        yesterday = today
        #print((bought_shares, money_left, yesterday, today))
        
    if bought_shares > 0:
        money_left += (bought_shares * yesterday)-7
    
    # Sell all shares at the end of investment period.
    # Return value of investment at the end of investment period
    return (tx, money_left)

def main():
    # Prompt user for ticker symbol (abbreviation of stock on NYSE).
    stock = input("Enter the ticker symbol for the stock: ")
    
    # Reads Microsoft stock data from Yahoo!Finances into variable d.
    # d contains date, open, high, low, etc. for one year. 
    d = {}
    d = pdr.get_data_yahoo(stock, '1/1/2015', '12/31/2015')
   
    # Calls function invest(d, m, n) for all combinations of m, n ϵ {1, 2, 3}
    output = []
    for m in range(1,4):
        for n in range(1,4):
            output.append((m, n, *invest(d, m, n)))
            
    print(pd.DataFrame(output, columns=["m","n","trans","Balance"]))
    # Collects balance returned by function invest into a 2D array or list with 
    # columns m, n, balance.

main()

# Do not increment transaction count on final day
# Don't deduct a final transaction fee for final cost
# First line of final output, in following format
#    m   n   Tran    Balance
#0   1   1   124     11473.31

In [ ]:
## SOLUTION provided by TAs Rakshesh

from pandas_datareader import data
import pandas as pd


def consecutive_check(conscarray):
    flag = True
    for i in range(len(conscarray)):
        if i < (len(conscarray) - 1):
            if (abs(conscarray[i] - conscarray[i + 1]) != 1):
                flag = False
    return flag


def invest(d, m, n):
    # initial investment
    capital = 10000
    transcation_fee = 7
    buying_price = d.Close[0]
    bought_shares = (capital - transcation_fee) / buying_price
    money_left = 0
    consecutive = 0
    own_share = True
    cons_array = []
    tx = 1

    for index, row in d.iloc[1:,:].iterrows():
        closing_price = row[3]

        if own_share:
            consecutive += 1
            if (closing_price < d.iloc[[d.index.get_loc(index) - 1]].iloc[0,3]):
                cons_array.append(consecutive)
            if (len(cons_array) >= m) & (consecutive_check(cons_array[-m:])):
                money_left = (bought_shares * closing_price - transcation_fee)  # sell all the stockts
                consecutive = 0
                cons_array = []
                tx += 1
                own_share = False
        else:
            consecutive += 1
            if (closing_price > d.iloc[[d.index.get_loc(index) - 1]].iloc[0,3]):
                cons_array.append(consecutive)
            if (len(cons_array) >= n) and (consecutive_check(cons_array[-n:])):
                bought_shares = (money_left - transcation_fee) / closing_price  # buy all the stockts
                money_left = 0
                own_share = True
                consecutive = 0
                tx += 1
                cons_array = []
    # If stocks are left, sell them at last closing price
    if money_left == 0:
        money_left = (bought_shares * d.Close[len(d) - 1] - 7)
        tx += 1

    return [money_left,tx]


def main():
    ticker= input('Enter the ticker Symbol: ')
    d = data.get_data_yahoo(ticker, '1/1/2015', '12/31/2015')
    m_values = [1, 2, 3]
    n_values = [1, 2, 3]
    value = pd.DataFrame(None, columns=['m', 'n', 'value', 'txn'])
    print('Executing....')
    for i in m_values:
        for j in n_values:
            value_tx = invest(d, i, j)
            money = value_tx[0]
            trnsctn = value_tx[1]
            value = value.append(pd.DataFrame([[i, j, money,trnsctn]], columns=['m', 'n', 'value', 'txn']),ignore_index=True)
    print(value)

main()

### Question 2 (10 points)
Use the best pair (m, n) from the Question 1 and use function invest(d, m, n) to simulate the investment strategy if you invest 10,000 in company Microsoft in year 2016.
Print the balance at the end of 2016.

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import pandas_datareader as pdr 
from pandas_datareader import data, wb

def invest(d, m, n):
    #Implements the investment algorithm, starting with capital of 10000
    capital = 10000    
    # Charged seven dollar fee on each transaction (buy / sell). 
    transaction_fee = 7    
    # Extract the closing values indexed by the date:
    d_Close = d["Close"]
    buying_price = d_Close[0]
    
    # Invest complete balance into X shares at beginning of time period.
    # Before you buy, first subtract transaction fee, then 
    # use resulting balance to buy as many shares as possible.    
    bought_shares = (capital - transaction_fee) / buying_price
    money_left = 0
    # Start transaction on second day
    tx = 1                 
    streak = 0
    yesterday = buying_price

    # Use for loop / iterate from second day w/ current iteration closing price    
    # for today in d_Close[1:]: 
    for index, row in d.iloc[1:, :].iterrows():  
        today = row[3]
        #days_down = 0
        #days_up = 0
        
        if today > yesterday and streak >= 0:
            streak += 1 
        elif today < yesterday and streak <= 0:
            streak -= 1       
        else:
            streak = 0
    
        if bought_shares > 0 and streak <= m:
            # check if stocks are down m consecutive days, and sell all shares.
            # After sell, subtract transaction fee from sell value to get balance.
            money_left += (bought_shares * today)-7
            tx = tx + 1
            bought_shares = 0
        
        elif bought_shares == 0 and streak >= n:
            # Check if stock is up m consecutive days, buy as many shares
            # If buy, subtract fee, buy as many shares as possible with balance.
            bought_shares += (money_left - 7) / today
            tx = tx + 1
            money_left = 0

        yesterday = today
        #print((bought_shares, money_left, yesterday, today))
        
    if bought_shares > 0:
        money_left += (bought_shares * yesterday)-7
    
    # Sell all shares at the end of investment period.
    # Return value of investment at the end of investment period
    return (tx, money_left)

def main():
    # Prompt user for ticker symbol (abbrev to identify stock on NYSE).
    stock = input("Enter the ticker symbol for the stock: ")    
    # Reads stock data from Yahoo!Finances into variable d.
    d = {}
    d = pdr.get_data_yahoo(stock, '1/1/2016', '12/31/2016')
    
    # Calls function invest(d, m, n) for best combinations of m, n ϵ {1, 2, 3}
    output = []
    m = 2
    n = 1
    output.append((m, n, *invest(d, m, n)))

    print(pd.DataFrame(output, columns=["m","n","trans","Balance"]))
    # Collects balance returned by function invest into a 2D array or list with 
    # columns m, n, balance.

main()

### Question 3 (30 points) Download 2016 stock data for company IBM, find and print:
• The longest time period in which IBM share value was up for the most consecutive days.
• The longest time period in which IBM share value was down for the most consecutive days.
Print both time periods with the values of the shares at the beginning and end of the periods.

#### Stock data
Import the “io.data” sub package to use pandas in accessing web data:
```
import pandas.io.data as web
```
For example, to get IBM stock data from Yahoo!Finances use the following statements:
```
d = {}
d = web.get_data_yahoo("IBM", '1/1/2017', '1/15/2017')
```
You will get the following data:
```
d
Out[95]:

           Open       High       Low        Close      Volume \
Date
2017-01-03 167.000000 167.869995 166.009995 167.190002 2934300
2017-01-04 167.770004 169.869995 167.360001 169.259995 3381400
2017-01-05 169.250000 169.389999 167.259995 168.699997 2682300
2017-01-06 168.690002 169.919998 167.520004 169.529999 2945500
2017-01-09 169.470001 169.800003 167.619995 167.649994 3184700
2017-01-10 167.979996 168.089996 165.339996 165.520004 4118700
2017-01-11 166.050003 167.759995 165.600006 167.750000 3599500
2017-01-12 167.770004 168.009995 165.559998 167.949997 2927500
2017-01-13 167.970001 168.479996 166.880005 167.339996 2875400

           Adj Close
Date
2017-01-03 165.878419
2017-01-04 167.932173
2017-01-05 167.376568
2017-01-06 168.200059
2017-01-09 166.334802
2017-01-10 164.221522
2017-01-11 166.434024
2017-01-12 166.632452
2017-01-13 166.027237
```

### Iterating through the stock data
An easy way to iterate through the stock data is to first convert the DataFrame into a Series. For example, we can extract the closing values indexed by the date:
```
d_close = d["Close"]
d_close
Out[34]:
Date
2017-01-03 167.190002
2017-01-04 169.259995
2017-01-05 168.699997
2017-01-06 169.529999
2017-01-09 167.649994
2017-01-10 165.520004
2017-01-11 167.750000
2017-01-12 167.949997
2017-01-13 167.339996

Name: Close, dtype: float64
```
Use “for” loop to iterate:
```
for i in d_close.index:
     print(d_close[i])
167.190002
169.259995
168.699997
169.529999
167.649994
165.520004
167.75
167.949997
167.339996
``` 

### Investment algorithm
You will explore an automated investment strategy which consists of following rules:
* If you make a decision to buy, you will always invest the complete balance to buy as many shares as possible of the (same) company X.
* If you make a decision to sell, you will always sell all shares that you have.
* Invest the complete balance into the X shares at the beginning of the time period.
* After the close of the market inspect if the shares are down or up compared to the previous close value.
* If you are invested (own the shares) and the shares are down for m consecutive days, sell all shares. You sell shares for the close value and you will have cash.
* If you are not invested (have cash) and the shares are up for n consecutive days, buy shares of company X for the price of close value.
* You are charged 7 dollar fee for each transaction (buy or sell). After you sell, subtract the fee from the sell value to get the balance. Before you buy, first subtract the transaction fee, and then use the resulting balance to buy as many shares as possible.
* You sell all shares at the end of the investment period.

For the sake of simplicity, assume that you will be able to buy or sell fractions of shares.

In [ ]:
import pandas as pd
#from pandas import Series, DataFrame
import pandas_datareader as pdr 
#from pandas_datareader import data, wb

def main():
    #  Download IBM stock data for 2016 from Yahoo!Finances into variable d.
    # d contains date, open, high, low, etc. for one year. 
    d = {}
    d = pdr.get_data_yahoo("IBM", '1/1/2016', '12/31/2016')
    
    # Extract the closing values indexed by the date:
    d_Close = d["Close"]
    buying_price = d_Close[0]
    
    # Start counting on second day
    yesterday = buying_price
    tx = 1                 
    streak = 0
    
    # Use for loop / iterate from second day w/ current iteration closing price    
    #for index, row in d.iloc[1:, :].iterrows():  
    for today in d_Close[1:]:        
        days_down = 0
        days_up = 0
        # longest time period IBM share down for most consecutive days.
        if today > yesterday:
            days_down += 1 
        
        # longest time period IBM value up for most consecutive days.
        elif today < yesterday and streak <= 0:
            days_up -= 1       
        
        #print((bought_shares, money_left, yesterday, today))    
   
    # Print both time periods with values of shares at beginning and end of periods.
    output = []
    output.append(("datae", "Closing_Price", "Up Days"))

    print(pd.DataFrame(output, columns=["date","Closing Price"]))
    # Collects balance returned by function invest into a 2D array or list with 

main()